In [1]:
import pandas as pd
import os
import numpy as np
import optuna
from sklearn.preprocessing import StandardScaler
# We are going to compare two different regression models here
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from optuna.integration import XGBoostPruningCallback
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import optuna
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Data preparation

In [2]:
# --------------------
# 1. Load and Preprocess
# --------------------
sfeatures = ['s' + str(i) for i in range(1, 22)]
col_names = ['unit_no', 'time', 'op1', 'op2', 'op3'] + sfeatures

data_train = pd.read_csv('./CMAPSSData/train_FD001.txt', sep=r'\s+', header=None, names=col_names)
data_test = pd.read_csv('./CMAPSSData/test_FD001.txt', sep=r'\s+', header=None, names=col_names)
RUL_labels = pd.read_csv('./CMAPSSData/RUL_FD001.txt', sep=r'\s+', header=None, names=['RUL'])

# Add RUL to train
max_times = data_train.groupby('unit_no')['time'].max().reset_index()
data_train = data_train.merge(max_times, on='unit_no', suffixes=('', '_max'))
data_train['RUL_calc'] = data_train['time_max'] - data_train['time']
data_train.drop(['time', 'time_max'], axis=1, inplace=True)

# Add RUL to test
RUL_labels['unit_no'] = RUL_labels.index + 1
max_times_test = data_test.groupby('unit_no')['time'].max().reset_index()
data_test = data_test.merge(max_times_test, on='unit_no', suffixes=('', '_max'))
data_test = data_test.merge(RUL_labels, on='unit_no')
data_test['RUL_calc'] = data_test['time_max'] - data_test['time'] + data_test['RUL']
data_test.drop(['time', 'time_max', 'RUL'], axis=1, inplace=True)

feature_cols = ['op1', 'op2', 'op3'] + sfeatures

In [3]:
# Normalize
scaler = StandardScaler()
data_train[feature_cols] = scaler.fit_transform(data_train[feature_cols])
data_test[feature_cols] = scaler.transform(data_test[feature_cols])
data_train.head()

,unit_no,op1,op2,op3,s1,s2,s3,s4,s5,s6,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL_calc
0,1,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-1.776357e-15,0.141683,...,-1.058890,-0.269071,-0.603816,-1.387779e-17,-0.781710,0.0,0.0,1.348493,1.194427,191
1,1,0.872722,-1.031720,0.0,0.0,-1.061780,0.211528,-0.643726,-1.776357e-15,0.141683,...,-0.363646,-0.642845,-0.275852,-1.387779e-17,-0.781710,0.0,0.0,1.016528,1.236922,190
2,1,-1.961874,1.015677,0.0,0.0,-0.661813,-0.413166,-0.525953,-1.776357e-15,0.141683,...,-0.919841,-0.551629,-0.649144,-1.387779e-17,-2.073094,0.0,0.0,0.739891,0.503423,189
3,1,0.324090,-0.008022,0.0,0.0,-0.661813,-1.261314,-0.784831,-1.776357e-15,0.141683,...,-0.224597,-0.520176,-1.971665,-1.387779e-17,-0.781710,0.0,0.0,0.352598,0.777792,188
4,1,-0.864611,-0.690488,0.0,0.0,-0.621816,-1.251528,-0.301518,-1.776357e-15,0.141683,...,-0.780793,-0.521748,-0.339845,-1.387779e-17,-0.136018,0.0,0.0,0.463253,1.059552,187


In [4]:
# --------------------
# 3. Proper Train/Validation Split by unit_no
# --------------------
train_ids, valid_ids = train_test_split(data_train['unit_no'].unique(), test_size=0.2, random_state=42)
train_df = data_train[data_train['unit_no'].isin(train_ids)]
valid_df = data_train[data_train['unit_no'].isin(valid_ids)]

# LSTM implementation

In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [6]:
class SequenceData(Dataset):
    def __init__(self, df, window_length = 30, feature_cols = None, label_col = 'RUL' ):
        self.window_length = window_length
        self.sequences = []
        self.labels = []
        self.unit_ids = []
        if feature_cols == None:
            feature_cols = [col for col in df.columns if col not in ['RUL', 'unit_no']]

        for unit_id, unit_data in df.groupby('unit_no'):
            unit_data = unit_data.reset_index(drop = True)
            for i in range(len(unit_data) - window_length + 1):
                window_data = unit_data.loc[i: i + window_length - 1]
                features = window_data[feature_cols].values
                labels = unit_data.loc[ i + window_length - 1, label_col]
                
                self.sequences.append(torch.tensor(features, dtype = torch.float32))
                self.labels.append(torch.tensor(labels, dtype = torch.float32))
                self.unit_ids.append(unit_id)
    def __len__(self):
        return len(self.sequences)
    def __getitem__(self, idx):
        return self.sequences[idx], self.labels[idx], self.unit_ids[idx]
    
class QuantileLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, output_len=1, quantiles=(0.1, 0.5, 0.9)):
        """
        Initialize the LSTM model for quantile forecasting.

        Args:
            input_size (int): Number of features per time step (1 for univariate).
            hidden_size (int): Number of hidden units in the LSTM.
            output_len (int): Number of future time steps to predict.
            quantiles (tuple): Quantiles to predict (e.g., 10th, 50th, 90th percentiles).
        """
        super().__init__()

        # Save the quantiles as a model attribute
        self.quantiles = quantiles

        # Define the LSTM layer
        # input_size: number of input features per time step
        # hidden_size: number of LSTM hidden units
        # batch_first=True: input/output tensors have shape (batch, seq, feature)
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)

        # Create a separate linear output head for each quantile
        # Each head maps the hidden state to a prediction of output_len size
        self.fc = nn.ModuleList([
            nn.Linear(hidden_size, output_len) for _ in quantiles
        ])

    def forward(self, x):
        """
        Forward pass through the model.

        Args:
            x (Tensor): Input tensor of shape (batch_size, seq_len, input_size)

        Returns:
            List[Tensor]: One tensor per quantile, each of shape (batch_size, output_len)
        """
        # Pass the input through the LSTM layer
        # hn is the final hidden state from the LSTM, shape: [1, batch_size, hidden_size]
        _, (hn, _) = self.lstm(x)

        # Remove the first dimension (num_layers), making hn shape: [batch_size, hidden_size]
        hn = hn.squeeze(0)

        # Pass the hidden state through each quantile-specific output head
        # Returns a list of output_len predictions for each quantile
        #return [layer(hn) for layer in self.fc]
        out = torch.cat([layer(hn).unsqueeze(1) for layer in self.fc], dim=1)
        return out.squeeze(-1) 

def quantile_loss(preds, target, quantiles):
    # preds: [batch_size, num_quantiles]
    # target: [batch_size]
    losses = []
    for i, q in enumerate(quantiles):
        errors = target - preds[:, i]
        loss = torch.max((q - 1) * errors, q * errors)
        losses.append(loss)
    total_loss = torch.mean(torch.stack(losses, dim=1).sum(dim=1))#.mean()
    return total_loss

In [7]:
data_train.head()

,unit_no,op1,op2,op3,s1,s2,s3,s4,s5,s6,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL_calc
0,1,-0.315980,-1.372953,0.0,0.0,-1.721725,-0.134255,-0.925936,-1.776357e-15,0.141683,...,-1.058890,-0.269071,-0.603816,-1.387779e-17,-0.781710,0.0,0.0,1.348493,1.194427,191
1,1,0.872722,-1.031720,0.0,0.0,-1.061780,0.211528,-0.643726,-1.776357e-15,0.141683,...,-0.363646,-0.642845,-0.275852,-1.387779e-17,-0.781710,0.0,0.0,1.016528,1.236922,190
2,1,-1.961874,1.015677,0.0,0.0,-0.661813,-0.413166,-0.525953,-1.776357e-15,0.141683,...,-0.919841,-0.551629,-0.649144,-1.387779e-17,-2.073094,0.0,0.0,0.739891,0.503423,189
3,1,0.324090,-0.008022,0.0,0.0,-0.661813,-1.261314,-0.784831,-1.776357e-15,0.141683,...,-0.224597,-0.520176,-1.971665,-1.387779e-17,-0.781710,0.0,0.0,0.352598,0.777792,188
4,1,-0.864611,-0.690488,0.0,0.0,-0.621816,-1.251528,-0.301518,-1.776357e-15,0.141683,...,-0.780793,-0.521748,-0.339845,-1.387779e-17,-0.136018,0.0,0.0,0.463253,1.059552,187


In [8]:
feature_cols = ['op1', 'op2', 'op3'] + sfeatures
feature_cols.append("RUL_calc")
feature_cols.append("unit_no")
print(feature_cols)

data_features_train = data_train[feature_cols]
data_features_train.head()

data_features_test = data_test[feature_cols]
data_features_test.head()

['op1', 'op2', 'op3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'RUL_calc', 'unit_no']


,op1,op2,op3,s1,s2,s3,s4,s5,s6,s7,...,s14,s15,s16,s17,s18,s19,s20,s21,RUL_calc,unit_no
0,1.055599,1.015677,0.0,0.0,0.678077,-0.853550,-1.191480,-1.776357e-15,0.141683,0.601408,...,-0.954235,-0.985107,-1.387779e-17,-0.781710,0.0,0.0,0.241943,0.774097,142,1
1,-1.230366,-1.031720,0.0,0.0,-1.941707,-0.338137,-1.501467,-1.776357e-15,0.141683,1.674769,...,-0.216648,-1.649034,-1.387779e-17,-0.136018,0.0,0.0,1.127183,0.941305,141,1
2,0.141213,0.333211,0.0,0.0,-0.441831,-0.584426,-0.843717,-1.776357e-15,0.141683,0.838677,...,-0.715712,0.052112,-1.387779e-17,-0.136018,0.0,0.0,1.459148,1.172256,140,1
3,1.924266,-0.008022,0.0,0.0,-0.481827,-1.044384,-0.279297,-1.776357e-15,0.141683,0.793483,...,-0.568929,-1.345067,-1.387779e-17,-1.427402,0.0,0.0,1.016528,0.775945,139,1
4,0.644125,-0.008022,0.0,0.0,-0.341839,-0.543650,-0.779276,-1.776357e-15,0.141683,0.895170,...,-0.745069,-1.041101,-1.387779e-17,-2.073094,0.0,0.0,0.961200,1.138999,138,1


In [9]:
train_dataset = SequenceData(data_features_train, label_col = 'RUL_calc')
train_loader = DataLoader(train_dataset, shuffle = False)

test_dataset = SequenceData(data_features_test, label_col = 'RUL_calc')
test_loader = DataLoader(test_dataset, shuffle = False)

In [ ]:
epochs = 30
quantiles = [0.05, 0.5, 0.95]

print(f'len(feature_cols) = {len(feature_cols)}')

model = QuantileLSTM(input_size=len(feature_cols)-1,quantiles = quantiles )
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()
for epoch in range(epochs): 
    epoch_loss = 0
    for x_batch, y_batch , _ in train_loader:
        optimizer.zero_grad()
        preds = model(x_batch)
        loss = quantile_loss(preds, y_batch, quantiles)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {epoch_loss / len(train_loader):.4f}")

len(feature_cols) = 26
Epoch 1: Loss = 24.8701
Epoch 2: Loss = 13.9634
Epoch 3: Loss = 14.2638
Epoch 4: Loss = 12.8398
Epoch 5: Loss = 13.7869
Epoch 6: Loss = 16.0616
Epoch 7: Loss = 24.5746


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

model.eval()

all_preds = []
all_truths = []

unit_preds = defaultdict(list)
unit_truths = defaultdict(list)

with torch.no_grad():
    for x_batch, y_batch, unit_ids in test_loader:
        preds = model(x_batch)  # [B, Q]
        #print(preds.shape)
        preds_stack = preds
        all_preds.append(preds.numpy())
        all_truths.append(y_batch.numpy())
        
        # Loop over each sample in the batch
        for i in range(x_batch.size(0)):
            # Extract unit ID — convert to integer if it's a tensor
            unit_id = unit_ids[i].item() if isinstance(unit_ids[i], torch.Tensor) else unit_ids[i]

            # Store the prediction (numpy array of quantiles) for this sample
            unit_preds[unit_id].append(preds_stack[i].cpu().numpy())

            # Store the corresponding true RUL value
            unit_truths[unit_id].append(y_batch[i].cpu().item())

            
# -------------------------------
# Convert lists to numpy arrays per unit for easy access and plotting
# -------------------------------
unit_preds = {u: np.array(p) for u, p in unit_preds.items()}
unit_truths = {u: np.array(t) for u, t in unit_truths.items()}

preds_np = np.vstack(all_preds)      # [N, 3]
truths_np = np.concatenate(all_truths)  # [N]

# Plot
plt.figure(figsize=(12, 6))
x_axis = np.arange(len(truths_np))
plt.plot(x_axis, preds_np[:, 1], label="Median Prediction", color='blue')
plt.fill_between(x_axis, preds_np[:, 0], preds_np[:, 2], color='gray', alpha=0.4, label='90% CI')
plt.plot(x_axis, truths_np, label="True RUL", color='black', linestyle='--')
plt.xlabel("Sample index")
plt.ylabel("RUL")
plt.legend()
plt.title("Quantile Predictions for RUL")
plt.tight_layout()
plt.show()

In [ ]:
lstm_ypred_05 = []
lstm_ypred_5 = []
lstm_ypred_95 = []
unit = []
RUL_true = []
for key, values in unit_preds.items(): 
    RUL_true.append(data_features_test[data_features_test['unit_no'] == key]['RUL_calc'].values[-1])
    unit.append(key)
    lstm_ypred_05.append(float(values[-1][0]))
    lstm_ypred_5.append(float(values[-1][1]))
    lstm_ypred_95.append(float(values[-1][2]))

In [ ]:
# Plot
plt.figure(figsize=(12, 6))
plt.scatter(RUL_true, lstm_ypred_05, label="Median Prediction", color='blue')
#plt.fill_between(x_axis, preds_np[:, 0], preds_np[:, 2], color='gray', alpha=0.4, label='90% CI')
#plt.plot(x_axis, truths_np, label="True RUL", color='black', linestyle='--')
plt.xlabel("RUL True")
plt.ylabel("RUL")
plt.legend()
plt.title("Quantile Predictions for RUL")
plt.tight_layout()
plt.show()